# 三体量子随机性认证 - Li2023论文复现（优化版）
本Notebook实现了基于Li等人2023年论文的三体量子随机性认证方案，并进行了性能优化。主要包括量子态生成、同态测量POVM构建、随机性认证SDP求解等核心功能。

In [1]:
import numpy as np  # 数学计算库
import cvxpy as cp  # 凸优化求解器
from qutip import Qobj, tensor, ptrace, qeye, num, ket2dm  # 量子工具箱
from numpy.polynomial.hermite import hermgauss, Hermite  # 厄米多项式相关函数
from math import factorial  # 阶乘函数
from itertools import zip_longest  # 迭代工具
import time  # 计时工具
from tqdm import tqdm  # 添加进度条  # 可视化进度

## 1. 三体量子态生成函数
生成S26型三体纠缠态，考虑了传输效率影响

In [2]:
def cv_state_S26(r, eta1, eta2, max_p):
    """
    生成三体连续变量纠缠态S26，考虑了传输效率的影响
    
    参数:
    - r: 压缩参数
    - eta1: 第一个分束器的传输效率
    - eta2: 第二个分束器的传输效率
    - max_p: 最大光子数截断
    
    返回:
    - psi: 三体量子态(Qobj对象)
    """
    # 计算压缩参数相关值
    t = np.tanh(r)  # 双曲正切函数，与压缩程度相关
    sqrt_eta1, sqrt_1_e1 = np.sqrt(eta1), np.sqrt(1-eta1)  # 计算传输效率的平方根
    d = max_p+1  # 希尔伯特空间维度
    amps = {}  # 存储各Fock态分量的振幅字典，键为(nA,nB,nC)三元组

    # 引入阶乘函数以提高代码可读性
    fac = factorial
    # 遍历所有可能的光子数p
    for p in range(max_p+1):
        # 计算p光子项的前置因子
        pref_p = (t**p) / ((2**p) * np.cosh(r))
        # 遍历k1参数
        for k1 in range(p+1):
            f_k1 = (2*eta1-1)**k1  # k1相关因子
            # 遍历k2参数
            for k2 in range(k1+1):
                f_bs1 = (4*sqrt_eta1*sqrt_1_e1)**(p-k1)  # 分束器相关因子
                # 计算组合数相关项
                num1 = fac(p + 2*k2 - k1) * fac(p - (2*k2 - k1))
                den1 = fac(p-k1)*fac(k1-k2)*fac(k2)
                sqrt1 = np.sqrt(num1/den1)

                # 计算各模式的光子数
                nA = p + (2*k2 - k1)
                nB1= p - (2*k2 - k1)

                # 遍历k3参数，考虑第二个分束器的影响
                for k3 in range(nB1+1):
                    # 二项式因子和第二个分束器相关项
                    c = fac(nB1) / (fac(k3)*fac(nB1-k3))
                    sqrt2 = np.sqrt(c * eta2**k3 * (1-eta2)**(nB1-k3))

                    # 计算符号因子
                    sign = (-1)**(p - (3*k2 + k3))
                    # 确定B和C模式的最终光子数
                    nB, nC = k3, nB1-k3

                    # 计算该Fock态分量的总振幅
                    amp = pref_p * f_bs1 * f_k1 * sqrt1 * sqrt2 * sign
                    # 将振幅累加到对应的Fock态上
                    amps[(nA,nB,nC)] = amps.get((nA,nB,nC),0) + amp

    # 构建态矢量
    vec = []
    # 遍历所有可能的光子数组合，按顺序构建态矢量
    for nA in range(d):
      for nB in range(d):
        for nC in range(d):
          vec.append(amps.get((nA,nB,nC),0.0))
    # 创建量子态对象并归一化
    psi = Qobj(np.array(vec), dims=[[d,d,d],[1,1,1]]).unit()
    return psi  # 返回生成的三体量子态

## 2. 同态测量POVM生成函数
创建周期性分箱的同态测量正算符值测度(POVM)

In [3]:
def make_homodyne_povm(N_bins, T, cutoff, Nq=200):
    """
    创建周期性分箱的同态测量POVM算子
    
    参数:
    - N_bins: 分箱数量
    - T: 周期长度
    - cutoff: Fock空间截断维度
    - Nq: 高斯-埃尔米特积分点数量
    
    返回:
    - Ms: POVM算子列表
    """
    d = cutoff + 1  # 希尔伯特空间维度

    # 获取高斯-埃尔米特求积节点和权重，用于数值积分
    x, w = hermgauss(Nq)

    # 构建Φ_n(x_i) = norm_n * H_n(x_i)，即归一化的厄米多项式
    Phi = np.empty((d, Nq))
    for n in range(d):
        Hn   = Hermite.basis(n)  # 获取物理学家定义的厄米多项式基
        # 计算归一化系数
        norm = 1.0 / (np.pi**0.25 * np.sqrt(2.0**n * factorial(n)))
        # 计算每个积分点上的多项式值
        Phi[n, :] = norm * Hn(x)

    # 周期性折叠并确定分箱索引
    tri    = T / N_bins  # 每个分箱的宽度
    xmod = x - np.floor(x / T) * T  # 将x映射到[0, T)区间
    idx  = np.floor(xmod / tri).astype(int)  # 确定每个点所属的分箱
    idx  = np.clip(idx, 0, N_bins - 1)  # 确保索引在有效范围内

    # 构建POVM算子
    Ms = []
    for b in range(N_bins):
        # 创建掩码，标记属于当前分箱的点
        mask = (idx == b).astype(float)
        # 计算加权权重
        W    = w * mask
        # 构建POVM算子矩阵
        mat  = (Phi * W) @ Phi.T  # 利用外积构建密度矩阵
        # 创建量子对象并添加到列表
        Ms.append(Qobj(mat, dims=[[d],[d]]))

    # 验证完备性：Σ_b M_b ≈ I
    I = sum(M.full() for M in Ms)
    # 降低断言的严格性，避免数值误差导致的错误
    assert np.allclose(I, np.eye(d), atol=1e-6), "Σ_b M_b ≠ I; try larger Nq."
    return Ms  # 返回构建的POVM算子列表

## 3. 随机性认证核心函数（优化版）
通过半定规划(SDP)求解Eve的最优猜测概率，进而计算最小熵

In [ ]:
def certify_randomness_ns(psi, Mbases, Mcases, solver_params=None):
    """
    优化版的随机性认证函数
    通过半定规划求解Eve的最优猜测概率，计算最小熵H_min
    
    参数:
    - psi: 三体量子态(Qobj对象)
    - Mbases: Bob的测量基，字典格式{'x': [...], 'p': [...]}
    - Mcases: Charlie的测量基，字典格式{'x': [...], 'p': [...]}
    - solver_params: 求解器参数字典，可以覆盖默认参数
    
    返回:
    - Hmin: 最小熵值
    - P_g: Eve的最优猜测概率
    - solve_time: 求解时间（秒）
    """
    # 获取本地希尔伯特空间维度
    d = psi.dims[0][0]           # local Hilbert dim (should be cutoff+1)
    # 将纯态转换为密度矩阵
    rho = ket2dm(psi)            # make it a density operator
    # 获取Bob和Charlie的测量结果数量
    nB = len(Mbases['x'])  
    nC = len(Mcases['x']) 

    # 1) 构建观测的量子态集合(assemblage)
    sigma_obs = {}
    # 遍历所有可能的测量设置(y,z)和结果(b,c)
    for y in ['x','p']:
    # Bob的测量设置：x或p正交
        for z in ['x','p']:
    # Charlie的测量设置：x或p正交
            for b in range(nB):
    # Bob的测量结果索引
                for c in range(nC):
    # Charlie的测量结果索引
                     # 获取对应的POVM算子
                     Mb, Mc = Mbases[y][b], Mcases[z][c]
                     # 构建联合测量算子: I ⊗ Mb ⊗ Mc
                     op = tensor(qeye(d), Mb, Mc)              # I ⊗ Mb ⊗ Mc
                     # 对Alice子系统进行偏迹，得到条件态
                     sigma_obs[(b,c,y,z)] = ptrace(op * rho, [0])  # on A

    # 2) 定义优化变量σ
    sigma = {}
    # 为所有可能的Eve猜测(e,e2)、测量设置(y,z)和结果(b,c)定义变量
    for e in range(nB):
    # Eve对Bob结果的猜测
        for e2 in range(nC):
    # Eve对Charlie结果的猜测
            for y in ['x','p']:
                for z in ['x','p']:
                    for b in range(nB):
                        for c in range(nC):
                            # 定义厄米矩阵变量
                            sigma[(e,e2,b,c,y,z)] = cp.Variable((d,d), hermitian=True)

    # 存储约束条件的列表
    constraints = []

    # 3) 约束条件：与观测到的量子态集合一致
    for (b,c,y,z), obs in sigma_obs.items():
        # 对所有Eve猜测求和必须等于观测到的条件态
        lhs = sum(sigma[(e,e2,b,c,y,z)] for e in range(nB) for e2 in range(nC))
        constraints.append(lhs == cp.Constant(obs.full()))

    # 4) 无信号条件约束 (根据论文中的式B6)
    for e in range(nB):
        for e2 in range(nC):
        # C边际分布与y无关
            for c in range(nC):
                  for z in ['x','p']:
                        # Bob使用x设置时的边际分布
                        lhs1 = sum(sigma[(e,e2,b,c,'x',z)] for b in range(nB))
                        # Bob使用p设置时的边际分布
                        lhs2 = sum(sigma[(e,e2,b,c,'p',z)] for b in range(nB))
                        # 两者必须相等
                        constraints.append(lhs1 == lhs2)
        # B边际分布与z无关
            for b in range(nB):
                  for y in ['x','p']:
                        # Charlie使用x设置时的边际分布
                        lhs1 = sum(sigma[(e,e2,b,c,y,'x')] for c in range(nC))
                        # Charlie使用p设置时的边际分布
                        lhs2 = sum(sigma[(e,e2,b,c,y,'p')] for c in range(nC))
                        # 两者必须相等
                        constraints.append(lhs1 == lhs2)

    # 5) 半正定约束：所有变量必须是半正定矩阵
    for var in sigma.values():
        constraints.append(var >> 0)

    # 6) 目标函数：最大化Eve的猜测概率
    # Eve同时猜对Bob和Charlie结果的概率之和
    obj_expr = cp.sum([
        cp.real(cp.trace(sigma[(e,e2,e,e2,'x','x')]))
        for e in range(nB) for e2 in range(nC)
   ])
    # 构建最大化问题
    prob = cp.Problem(cp.Maximize(obj_expr), constraints)

    # 默认求解器参数 - 优化版设置
    default_params = {
        'solver': cp.SCS,  # 使用SCS求解器
        'verbose': False,  # 不显示详细求解过程
        'max_iters': 5000,  # 减少迭代次数，提高速度
        'eps': 1e-3,        # 放宽精度要求，提高速度
        'acceleration_lookback': 10,  # 添加加速策略
        'rho_x': 1e-3       # 调整rho参数以提高收敛性
    }

    # 合并用户提供的参数
    if solver_params:
        default_params.update(solver_params)
    
    try:
        # 记录求解时间
        start_time = time.time()
        # 求解优化问题
        result = prob.solve(**default_params)
        solve_time = time.time() - start_time
        
        # 获取最优猜测概率
        P_g  = float(prob.value)
        # 确保概率在有效范围内，防止数值误差
        P_g  = float(np.clip(P_g, 0.0, 1.0))
        # 计算最小熵：H_min = -log2(P_g)
        Hmin = 0.0 if P_g <= 0 else max(0.0, -np.log2(P_g))
        
        # 如果求解精度不够，给出警告但继续执行
        if prob.status != cp.OPTIMAL:
            print(f"警告：求解状态为 {prob.status}，结果可能不够准确")
        
        return Hmin, P_g, solve_time
    except Exception as e:
        print(f"求解器错误: {e}")
        # 出现错误时返回默认值，避免程序中断
        return 0.0, 1.0, 0.0  # 错误情况下的默认返回值

## 4. 主程序：扫描参数并计算最小熵
使用优化策略扫描不同eta2值，计算对应的最小熵，复现论文结果

In [ ]:
if __name__ == "__main__":
    """
    主程序入口，执行三体量子随机性认证的完整流程
    扫描不同的eta2值，计算对应的最小熵H_min
    """
    # 优化版参数 - 减少计算量
    r      = 0.345  # 压缩参数
    eta1   = 0.5    # 第一个分束器效率
    p_max  = 1      # 最大光子数截断
    cutoff = 1      # Fock空间截断维度
    d      = cutoff + 1  # 希尔伯特空间维度
    N_bins = 4      # 分箱数量
    # 减少搜索点数量，从9个减少到5个，提高计算速度
    T_B_vals = np.linspace(2, 10, 5)   # Bob的周期参数扫描范围
    T_C_vals = np.linspace(2, 10, 5)   # Charlie的周期参数扫描范围
    Nq     = 120  # 减少高斯-埃尔米特点数量，提高速度
    
    # 创建相空间旋转算符，用于从x正交转换到p正交
    R_pi2 = (-1j * (np.pi/2) * num(d)).expm()
    
    # 存储每个eta2值对应的最佳H_min
    Hmins = []
    total_time = 0  # 记录总计算时间
    
    # 使用tqdm添加进度条，可视化eta2扫描进度
    eta2_values = np.linspace(0, 1, 11)  # eta2从0到1，共11个点
    for i, eta2 in enumerate(tqdm(eta2_values, desc="处理eta2参数")):
        best_H = 0.0  # 初始化最佳最小熵
        
        # 生成三体量子态
        psi = cv_state_S26(r, eta1, eta2, p_max)
        
        # 减少嵌套循环的深度 - 提前判断是否有希望得到更好的结果
        # 记录已处理的组合数
        combinations_processed = 0
        max_combinations = len(T_B_vals) * len(T_C_vals)
        early_stop_threshold = max_combinations * 0.7  # 70%组合后如果没有改进可以提前停止
        last_improvement = 0
        
        # 遍历Bob的周期参数
        for TxB in T_B_vals:
            TpB = 2*np.pi*N_bins / TxB  # 计算p正交的周期参数
            
            # 生成Bob的POVMs
            Mb_x = make_homodyne_povm(N_bins, TxB, cutoff, Nq=Nq)  # x正交POVM
            Mb_p_xform = make_homodyne_povm(N_bins, TpB, cutoff, Nq=Nq)  # p正交POVM（未旋转）
            # 通过旋转算符将x正交POVM转换为p正交POVM
            Mb_p = [R_pi2.dag() * M * R_pi2 for M in Mb_p_xform]
            
            # 遍历Charlie的周期参数
            for TxC in T_C_vals:
                combinations_processed += 1
                
                # 早停机制：如果长时间没有改进且已处理大部分组合，提前停止
                if combinations_processed > last_improvement + early_stop_threshold:
                    print(f"  早停: eta2={eta2:.2f}，长时间无改进")
                    break
                
                TpC = 2*np.pi*N_bins / TxC  # 计算p正交的周期参数
                
                # 生成Charlie的POVMs
                Mc_x = make_homodyne_povm(N_bins, TxC, cutoff, Nq=Nq)  # x正交POVM
                Mc_p_xform = make_homodyne_povm(N_bins, TpC, cutoff, Nq=Nq)  # p正交POVM（未旋转）
                # 通过旋转算符将x正交POVM转换为p正交POVM
                Mc_p = [R_pi2.dag() * M * R_pi2 for M in Mc_p_xform]
                
                # 使用优化的求解器参数
                solver_params = {
                    'eps': 1e-3,  # 放宽精度，提高速度
                    'max_iters': 5000  # 减少最大迭代次数
                }
                
                # 对于中间值可以使用更激进的参数，两端值可以更精确
                if i in [0, len(eta2_values)-1]:
                    solver_params['eps'] = 1e-4  # 两端点使用更高精度
                
                # 执行随机性认证计算
                Hmin, Pg, solve_time = certify_randomness_ns(
                    psi,
                    {'x': Mb_x, 'p': Mb_p},
                    {'x': Mc_x, 'p': Mc_p},
                    solver_params=solver_params
                )
                
                total_time += solve_time
                
                # 更新最佳值和最后改进的位置
                if Hmin > best_H:
                    best_H = Hmin
                    last_improvement = combinations_processed
                    
        # 保存当前eta2对应的最佳最小熵
        Hmins.append(best_H)
        # 打印进度信息
        print(f"eta2={eta2:.2f}: 最佳H_min={best_H:.5f}")
    
    # 输出总计算时间和最终结果
    print(f"总计算时间: {total_time:.2f} 秒")
    print(f"最终Hmins结果: {Hmins}")

处理eta2参数:   9%|▉         | 1/11 [02:54<29:06, 174.63s/it]


  早停: eta2=0.00，长时间无改进
eta2=0.00: 最佳H_min=0.15576


处理eta2参数:  18%|█▊        | 2/11 [05:23<23:57, 159.76s/it]


eta2=0.10: 最佳H_min=0.00732


处理eta2参数:  27%|██▋       | 3/11 [07:49<20:27, 153.48s/it]


...

eta2=0.80: 最佳H_min=0.00657


处理eta2参数:  91%|█████████ | 10/11 [24:35<02:25, 145.02s/it]


eta2=0.90: 最佳H_min=0.00732


处理eta2参数: 100%|██████████| 11/11 [28:10<00:00, 153.67s/it]


eta2=1.00: 最佳H_min=0.15576

总计算时间: 1641.30 秒
最终Hmins结果: [np.float64(0.1557629557794112), np.float64(0.007320450433710241), np.float64(0.006567463440057703), np.float64(0.006184966129671333), np.float64(0.006324025396373586), np.float64(0.0063481189180133995), np.float64(0.006324025403448868), np.float64(0.0061849661317877605), np.float64(0.006567463438641764), np.float64(0.007320450429592538), np.float64(0.1557629557857033)]
